In [1]:
import sys
import pickle
import json
import re
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [2]:
def feature_transform(train_file, words, tags):
    
    words_len = len(words)
    tags_len = len(tags)
    row_idx = []
    column_idx = []
    data = []
    y = []
    ctr = 0
    feature_length = words_len + tags_len # Per head

    f = open(train_file)

    for line in f:
            if(line.rstrip()):
                line = re.sub("\s+"," ",line)
                line1 = line.split(";")

                a1 = line1[0].split(" ")
                a2 = line1[1].split(" ")
                a3 = line1[2].split(" ")
                a4 = line1[3].strip()
                
                if a3[1] != "U":
                    
                    ctr += 1
                    
                    if(a1[0] == "H"):
                        column_idx.append(words.index(a1[1]))
                        column_idx.append(tags.index(a1[4]) + words_len)

                    elif(a1[0] == "ROOT"):
                        column_idx.append(words.index("ROOT"))
                        column_idx.append(tags.index("ROOT") + words_len)

                    row_idx += [ctr-1]*4
                    data += [1] *4
                    column_idx.append(feature_length + words.index(a2[2]))
                    column_idx.append(feature_length + tags.index(a2[5]) + words_len)

                    y.append(a4)

    f.close()

    X = csr_matrix((data, (row_idx, column_idx)), shape=(ctr,2*(words_len+tags_len)))
    
    
    return X, y

In [3]:
listfile = "data_tokens.json"
f = open(listfile)
data = json.load(f)
f.close()

words = data["words"]
tags = data["tags"]

train_file = 'training_data.txt'
test_file = "testing_data.txt"

X_train, y_train = feature_transform(train_file, words, tags)
X_test, y_test = feature_transform(test_file, words, tags)


model = LinearSVC()
model.fit(X_train, y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [4]:
print(classification_report(y_train, pred_train))

/home/ayan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         K2U       0.00      0.00      0.00         1
        ROOT       1.00      1.00      1.00     11145
       UNDEF       1.00      1.00      1.00         1
         adv       0.85      0.87      0.86      1325
        ccof       0.91      0.99      0.95     12223
         enm       1.00      1.00      1.00         2
      fragof       1.00      0.56      0.71         9
       jjmod       0.98      0.99      0.98       716
         jk1       1.00      0.50      0.67        10
          k1       0.75      0.85      0.80     13000
         k1s       0.83      0.74      0.78      1487
         k1u       1.00      0.39      0.56        46
          k2       0.79      0.78      0.78     10234
         k2g       0.86      0.39      0.53        31
         k2p       0.82      0.78      0.80       387
         k2s       0.88      0.56      0.69       296
         k2u       1.00      0.47      0.64        34
          k3       0.81    

In [5]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

         K7a       0.00      0.00      0.00         1
        ROOT       1.00      1.00      1.00      1432
       UNDEF       0.00      0.00      0.00         0
         adv       0.81      0.78      0.79       147
        ccof       0.87      0.96      0.91      1558
      fragof       0.00      0.00      0.00         0
       jjmod       0.93      0.93      0.93        80
          k1       0.59      0.76      0.67      1584
         k1s       0.75      0.57      0.65       210
         k1u       0.00      0.00      0.00         9
          k2       0.66      0.62      0.64      1369
         k2g       0.00      0.00      0.00         0
         k2p       0.38      0.15      0.21        40
         k2s       0.29      0.07      0.11        29
         k2u       0.00      0.00      0.00         1
          k3       0.21      0.07      0.11        54
          k4       0.48      0.36      0.41       197
         k4a       0.30    

/home/ayan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
